# Pre-processing of Data Sets

In [1]:
import pandas as pd

### Utility Data

In [2]:
dfu = pd.read_csv('data/ExampleDataExport.csv')
dfu.head()

,Site ID,Site Name,Vendor Code,Vendor Name,Account Number,Bill Date,Due Date,Entry Date,Invoice #,Voucher #,From,Thru,Service Name,Item Description,Meter Number,Usage,Cost,Units,Account Financial Code,Site Financial Code
0,ANSBG1,FNSB-Animal Control,VF314940,Fairbanks Natural Gas,10282 (2408 DAVIS),8/30/2010,8/30/2010,1/24/2011,NaN,NaN,7/29/2010,8/30/2010,Natural Gas,Natural gas (CCF),NaN,2716.0,6222.36,CCF,61837,NaN
1,ANSBG1,FNSB-Animal Control,VF314940,Fairbanks Natural Gas,10282 (2408 DAVIS),8/30/2010,8/30/2010,1/24/2011,NaN,NaN,7/29/2010,8/30/2010,Natural Gas,Regulatory Cost Charge,NaN,NaN,21.53,NaN,61837,NaN
2,ANSBG1,FNSB-Animal Control,VF314940,Fairbanks Natural Gas,10282 (2408 DAVIS),8/30/2010,8/30/2010,1/24/2011,NaN,NaN,7/29/2010,8/30/2010,Natural Gas,Customer Charge,NaN,NaN,17.25,NaN,61837,NaN
3,ANSBG1,FNSB-Animal Control,VF314940,Fairbanks Natural Gas,10282 (2408 DAVIS),9/30/2010,9/30/2010,1/24/2011,NaN,NaN,8/30/2010,9/30/2010,Natural Gas,Natural gas (CCF),NaN,3526.0,8078.07,CCF,61837,NaN
4,ANSBG1,FNSB-Animal Control,VF314940,Fairbanks Natural Gas,10282 (2408 DAVIS),9/30/2010,9/30/2010,1/24/2011,NaN,NaN,8/30/2010,9/30/2010,Natural Gas,Regulatory Cost Charge,NaN,NaN,27.93,NaN,61837,NaN


In [20]:
cols = ['Site ID', 'Vendor Code', 'Vendor Name', 'Account Number', 'Service Name', 'Item Description',
       'Meter Number', 'Units', 'Account Financial Code', 'Site Financial Code']
for col in cols:
    print('{0:24s}: {1}'.format(col, list(dfu[col].unique())))

Site ID                 : ['ANSBG1']
Vendor Code             : ['VF314940', 'VG372746', 'VG354933']
Vendor Name             : ['Fairbanks Natural Gas', 'Golden Heart Utilities', 'Golden Valley Electric']
Account Number          : ['10282  (2408 DAVIS)', '1311001100', '31850']
Service Name            : ['Natural Gas', 'Water', 'Sewer', 'Electricity']
Item Description        : ['Natural gas (CCF)', 'Regulatory Cost Charge', 'Customer Charge', 'Gas Charge (CCF)', 'Late charge', 'Water Usage (Gallons)', 'Sewer Usage (Gallons)', 'Water Fixed Charge', 'Sewer Fixed Charge', 'Fire Protection', 'Other charges', 'Misc. credit', 'Meter charge', 'Customer Charge - Water', 'Customer Charge - Sewer', 'Plant Replacement ADJ', 'Cost of Energy Adjustmen', 'Energy charge', 'KW Charge', 'Fuel Adjustment', 'On peak demand', 'Fuel cost adjustment', 'Fuel & Purchased Power', 'Utility Charge']
Meter Number            : [nan, 89672.0]
Units                   : ['CCF', nan, 'Gallons', 'kWh', 'kW']
Account Fina

### Other Building Data

In [52]:
df_bldg = pd.read_excel('data/Other_Building_Data.xlsx', sheetname='Building', skiprows=3, index_col='site_ID')
print(df_bldg.loc['ANSBG1'])
df_bldg.head()

site_name        Animal Control
address         2408 Davis Road
city                  Fairbanks
primary_func     Animal Shelter
year_built                 1993
sq_ft                     14000
onsite_gen                  NaN
dd_site                    PAFA
Name: ANSBG1, dtype: object


,site_name,address,city,primary_func,year_built,sq_ft,onsite_gen,dd_site
site_ID,,,,,,,,
ANSBG1,Animal Control,2408 Davis Road,Fairbanks,Animal Shelter,1993,14000,NaN,PAFA
BIGDIP,Big Dipper Ice Arena,1920 Lathrop Street,Fairbanks,Ice Arena,1990,23000,NaN,PAFA


### Degree Days

In [37]:
df_dd = pd.read_excel('data/Other_Building_Data.xlsx', sheetname='Degree Days', skiprows=3, parse_dates=['Month'])
df_dd['year'] = [d.year for d in df_dd.Month]
df_dd['mo'] = [d.month for d in df_dd.Month]
df_dd.set_index(['year', 'mo'], inplace=True)
df_dd.drop('Month', axis=1, inplace=True)

print(df_dd.loc[(2017, 7), 'PAFA'])
df_dd.tail()

86


PAFA
year mo      
2017 4    863
     5    448
     6    135
     7     86
     8    248

### Fuel Information

In [53]:
df_fuel = pd.read_excel('data/Other_Building_Data.xlsx', sheetname='Fuel Types', skiprows=3, index_col=[0, 1])
print(df_fuel.loc[('Electricity', 'kWh'), 'btu_per_unit'])
df_fuel

3412


,,btu_per_unit
fuel,unit,
Natural Gas,CCF,102000
Electricity,kWh,3412
Fuel Oil,Gallons,135000
